## Homework 5 SVM problem

Load all modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

#### Part A

Data Pre-Proceing 
Create a binary label based on the column “Chance of Admit”. Convert any values bigger than the mean to 1 and 0 otherwise.

In the following code block I load the data read in the Admission data into a data frame and remove any rows with missing values. I then select the Label column "Chance of Admit" and load it into to my 'y' data frame, which I then convert to binary 1 or 0 using the averge of the "Chance of Admit" Column as the decsion boundary.

In [ ]:
# Load Data Frame
df = pd.read_csv('Admission.csv')

# drop empty rows
df.dropna(inplace=True)
# print first 10 rows and shape
print(df.head(10))
print("Data shape: ", df.shape)

# Find mean of Chance of Admit, note the space after admit in the data
mean_chance_of_admit = df['Chance of Admit '].mean()
print("Mean Chance of admit: ", mean_chance_of_admit)

# Setup label data frame
y = df[['Chance of Admit ']]
# If greater than mean chance of admit set to 1 otherwise set to 0
y['Chance of Admit '] = np.where(y['Chance of Admit '] > mean_chance_of_admit, 1, 0)
print(y.head(5))


#### Part B

Model Initialization: Initialize 4 different SVM models with the following kernels.

In the following code block I initialize all of the requested SVM models using scikit-learn.


In [ ]:
# Import SVC and LinearSVC from sklearn
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

# SVM regularization parameter
C = 1.0

# Initialize SVM models with different kernels
svm_linear = SVC(kernel='linear', C=C)
linear_svc = LinearSVC(C=C)
svm_rbf = SVC(kernel='rbf', C=C)
svm_poly = SVC(kernel='poly', degree=3, C=C)

#### Part C and D

Feature Selection and Model Training: Train each SVM Model above with the following feature combinations to predict admission.

Result Visualization: Visualize the decision boundary for each model and for each input combination.

In the following code block I get the feature subsets from the main dataframe. I then train the models on each feature subset and the results. Lastly I plot the decsion boundry for the model.

In [ ]:

# Define feature subsets

# Subset containing CGPA and SOP columns
X_cgpa_sop = df[['CGPA','SOP']]
# Subset containing CGPA and GRE Score columns
X_cgpa_gre = df[['CGPA', 'GRE Score']]
# Subset containing SOP and LOR columns
X_sop_lor = df[['SOP', 'LOR ']]
# Subset containing LOR and GRE Score columns
X_lor_gre = df[['LOR ', 'GRE Score']]

X_cgpa_sop.Name = "GPA and SOP"
X_cgpa_gre.Name = "GPA and GRE"
X_sop_lor.Name = "SOP and LOR"
X_lor_gre.Name = "LOR and GRE"

# Create a list of subsets
datasets = [X_cgpa_sop, X_cgpa_gre, X_sop_lor, X_lor_gre]

# Convert result dataframe to numpy matrix
ynp = np.array(y.values, dtype=np.float64)
# Flatten to array
ynp = ynp.flatten()

fig, axes = plt.subplots(4, 4, figsize=(20, 20))

for i, curr_set in enumerate(datasets):
    for j, clf in enumerate((svm_linear, linear_svc, svm_rbf, svm_poly)):
        # Convert feature dataframe to numpy matrix
        Xnp = np.array(curr_set.values, dtype=np.float64)

        # Fit SVM model
        clf.fit(Xnp, ynp)

        # step size in the mesh
        h = .02 
         
        # Define meshgrid for plotting decision boundary
    
        # calculates the minimum and maximum values of the first feature
        x_min, x_max = Xnp[:, 0].min() - 1, Xnp[:, 0].max() + 1
        # calculates the minimum and maximum values of the second feature
        y_min, y_max = Xnp[:, 1].min() - 1, Xnp[:, 1].max() + 1
        # This creates a meshgrid for the decision boundary plot
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        # predicts the class labels for each point in the meshgrid 
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
        # match the shape of the meshgrid
        Z = Z.reshape(xx.shape)


        # Swtich subplot
        ax = axes[i, j]
        # Plot decision boundary
        ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
        # Plot training points
        ax.scatter(Xnp[:, 0], Xnp[:, 1], c=ynp, cmap=plt.cm.coolwarm)
        # Set x-axis label
        ax.set_xlabel(curr_set.columns.tolist()[0])
        # Set y-axis label
        ax.set_ylabel(curr_set.columns.tolist()[1])
        # Set x-axis limits
        ax.set_xlim(xx.min(), xx.max())
        # Set y-axis limits
        ax.set_ylim(yy.min(), yy.max())
        # Set title
        if clf == svm_linear:
            ax.set_title(f'Decision Boundary for {curr_set.columns.tolist()[0]} and {curr_set.columns.tolist()[1]} with svm linear')
        if clf == linear_svc:
            ax.set_title(f'Decision Boundary for {curr_set.columns.tolist()[0]} and {curr_set.columns.tolist()[1]} with linear svc')
        if clf == svm_rbf:
            ax.set_title(f'Decision Boundary for {curr_set.columns.tolist()[0]} and {curr_set.columns.tolist()[1]} with svm rbf')
        if clf == svm_poly:
            ax.set_title(f'Decision Boundary for {curr_set.columns.tolist()[0]} and {curr_set.columns.tolist()[1]} with svm poly')

# Plot
plt.tight_layout()
plt.show()

#### Part E

Result Analysis: Just by looking at the figures you generated before, answer this question: Which of the feature + kernel combinations gave you the best result?

I think the linear svm model with SOP and LOR barely beat out the linear svc model with the same features for the best spot.


#### Part F

Result Post-processing: Were there any outliers in the data? If yes, please explain how we can use a one-class SVM to detect them

Yes, since SVM basically creates a boundary based off the majority of the classification data we can label any falsely classified data as an outlier since it deviates from majority of the data.